# Heteroskedasticity (이분산)
- 작성자: 고려대학교 경제학과 한치록 교수
## 도입

매뉴얼 `2-01`과 `2-02`에서 [OLS에 대하여 소개하고][3_01] [견고한 표준오차 계산][3_02]에 대하여 살펴보았다.

이분산([heteroskedasticity][het])이란 오차항의 분산이 설명변수 값에 의존하는 경우를 의미한다. 설명변수들의 값이 관측치마다 다를 수 있으므로 오차 분산이 관측치마다 다른 현상을 이분산이라 하기도 한다. 오차항이 이분산적이면 OLS 추정량의 비편향성이나 consistency에는 문제가 없으나, OLS 추정량의 분산식이 달라지고, 따라서 OLS를 이용하여 추론(검정 등)을 할 때 통상적인 표준오차를 사용하는 것이 타당성을 잃는다. [앞에서는][3_02] OLS 추정법을 사용할 때 올바른 추론을 하기 위한 방법(견고한 표준오차를 사용하는 방법)을 이야기하였다. 이 글에서는 이분산과 관련된 다른 문제들을 살펴본다. 내용은 다음과 같다.

1. 이분산 검정
2. 가중최소제곱법([weighted least squares](https://en.wikipedia.org/wiki/Weighted_least_squares), WLS)
3. 가중치를 추정하는 WLS

[3_01]: 01%20Least%20Squares.ipynb
[3_02]: 02%20Robust%20Inferences%20with%20OLS.ipynb
[sm]: https://www.statsmodels.org/
[sm_wls]: https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.WLS.html
[smf_wls]: https://www.statsmodels.org/dev/generated/statsmodels.formula.api.wls.html
[het]: https://en.wikipedia.org/wiki/Homoscedasticity_and_heteroscedasticity

## 이분산 검정

이분산은 오차 분산이 설명변수 값에 의존하는 현상을 나타내므로 검정을 위해서는 OLS 잔차의 제곱(분산은 제곱의 평균이므로)을 설명변수들의 함수에 대하여 회귀하고 모든 설명변수의 계수가 0인지 검정하면 된다. [Breusch and Pagan (1979)의 검정](https://en.wikipedia.org/wiki/Breusch–Pagan_test)과 [White (1980)의 검정](https://en.wikipedia.org/wiki/White_test)이 많이 사용된다.

### Breusch-Pagan 검정
예를 들어 설명하자. 회귀모형을 $y=\beta_0 + \beta_1 x_1 + \beta_2 x_2 + u$라 하고 $var(u|x_1,x_2) = \delta_0 + \delta_1 x_1 + \delta_2 x_2$라 한다면 OLS 잔차 $\hat{u}$의 제곱 $\hat{u}^2$을 $x_1$과 $x_2$에 대하여 회귀한 후 $x_1$과 $x_2$의 계수가 0이라는 것을 검정한다. 단, Breusch and Pagan (1979) 논문에서는 [LM 검정][lm]을 이용한다. [R][R]에서 Breusch-Pagan 검정은 `lmtest`에 `bptest`로 구현되어 있다. 파이썬의 Statsmodels에는 [statsmodels.stats.diagnostic.het_breuschpagan][sm_bp]으로 구현되어 있다.

Breusch-Pagan 검정을 실험해 보자. `Death.csv` 파일의 2010년 데이터를 이용한 분석에서 이분산이 존재하는지 살펴본다. 데이터셋은 다음과 같다.

[R]: https://r-project.org/
[lm]: https://en.wikipedia.org/wiki/Score_test
[sm_bp]: https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.het_breuschpagan.html

In [1]:
import pandas as pd

df = pd.read_csv('data/Death.csv')
df10 = df[df.year==2010].copy()
df10

,region,year,regpop,death,drink,smoke,aged,vehipc,deathrate
2,40,2010,102557,624,53.0,22.5,12.69,0.37,6.084421
5,39,2010,178899,966,55.6,25.6,9.90,0.41,5.399695
8,42,2010,67104,717,41.3,20.9,23.86,0.40,10.684910
11,43,2010,18739,149,50.7,28.8,19.32,0.42,7.951332
14,41,2010,201000,1007,56.4,25.3,9.92,0.43,5.009950
...,...,...,...,...,...,...,...,...,...
245,22,2010,51509,575,47.1,22.8,25.94,0.37,11.163100
248,19,2010,35591,443,49.9,25.6,29.31,0.41,12.446970
251,24,2010,41197,469,46.6,19.8,27.49,0.43,11.384320
254,16,2010,63421,688,46.0,24.4,22.21,0.37,10.848140


OLS 추정을 하고 회귀오차에 이분산이 존재하는지 검정하는 방법은 다음과 같다.

In [2]:
import statsmodels.formula.api as smf

ols = smf.ols("deathrate ~ smoke + aged + vehipc", data = df10).fit()

# https://www.geeksforgeeks.org/how-to-perform-a-breusch-pagan-test-in-python/
import statsmodels.stats.api as sms

lm,p_lm,f,p_f = sms.het_breuschpagan(ols.resid, ols.model.exog)
(lm,p_lm)

(0.22806974664980517, 0.9729355149825276)

위 검정에서 LM 검정통계값은 약 0.228, p값은 약 0.97로서 매우 크므로 오차항이 등분산적이라는 귀무가설이 기각되지 않는다.

참고로, R을 사용한 결과는 다음과 같으며, 위에서 구한 LM 검정 결과와 동일하다.
```r
# R codes
library(lmtest)
DF <- read.csv('../data/Death.csv')
DF10 <- subset(DF, year==2010)
bptest(deathrate ~ smoke + aged + vehipc, data=DF10)

#     studentized Breusch-Pagan test
#
# data:  deathrate ~ smoke + aged + vehipc
# BP = 0.22807, df = 3, p-value = 0.9729
```

### White 검정

[White (1980)의 검정](https://en.wikipedia.org/wiki/White_test)은 모든 설명변수와 제곱항, 교차항을 검정용 회귀식 우변에 사용한다. White 검정은 [statsmodels.stats.diagnostic.het_white](https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.het_white.html)로 구현되어 있다. 원래 `het_white`는 LM stat, p-value, F-stat, p-value를 리포트하지만, 결과를 온전히 report하기 위해서는 자유도가 필요하여 5번째 리턴 원소로 Model DF를 리턴한다.

In [3]:
import bok_da as bd

In [4]:
# https://www.einblick.ai/python-code-examples/white-heteroskedasticity-test-python/
lm,p_lm,f,p_f,_ = sms.het_white(ols.resid, ols.model.exog)
(lm,p_lm)

(2.9695565787496747, 0.9654897064067687)

R에서는 `bptest` 명령을 사용하면서 검정을 위한 우변에 모든 변수들을 포함시키는 방식을 사용한다. 다음 R 결과를 보라. 위의 파이썬 결과와 동일하다.
```r
# https://www.statology.org/white-test-in-r/
bptest(deathrate~smoke+aged+vehipc, ~(smoke+aged+vehipc)^2+I(smoke^2)+I(aged^2)+I(vehipc^2), data=DF2010)

#     studentized Breusch-Pagan test
#
# data:  deathrate ~ smoke + aged + vehipc
# BP = 2.9696, df = 9, p-value = 0.9655
```

### 단순화된 White 검정

White 검정에서는 변수의 개수가 많으면 검정용 회귀식의 우변변수 개수가 급속히 증가한다. 정확한 숫자는 $k+k+k(k-1)/2 = k(k+3)/2$개이다. 표본크기가 크지 않으면 자유도가 너무 작아지므로 X 변수들, 제곱항, 상호작용항들을 모두 포함하는 대신에 $\hat{y}$와 $\hat{y}^2$ 항을 사용할 수도 있다. 이것이 단순화된 White 검정이다.

In [5]:
import statsmodels.api as sm

# Simplified White test
lm,p_lm,f,p_f,_ = sms.het_white(ols.resid, sm.add_constant(ols.fittedvalues))
(lm,p_lm)

(0.5073415779205672, 0.7759472103904437)

R 결과도 동일하다.
```r
# R simplified White test
fm <- deathrate ~ smoke + aged + vehipc
DF10[['yhat']] <- lm(fm, data=DF10)$fit
bptest(fm, ~yhat+I(yhat^2), data=DF10)

#     studentized Breusch-Pagan test
#
# data:  deathrate ~ smoke + aged + vehipc
# BP = 0.50734, df = 2, p-value = 0.7759
```

## 가중최소제곱법(WLS)

이분산([heteroskedasticity][het])이 있는 경우 OLS는 BLUE가 아니며, $var(u_i) = \sigma^2 h_i$일 때 $1/h_i$를 가중치로 하는 가중최소제곱법([weighted least squares](https://en.wikipedia.org/wiki/Weighted_least_squares), WLS)이 BLUE이다. 이하에서는 WLS 구현법을 알아 본다.

### 인터페이스

WLS를 구현하려면 파이썬 코딩 시 "OLS"가 들어갈 자리에 "WLS"가 들어간다고 생각하면 기억하기 쉽다. [Statsmodels][sm]에서는 OLS와 마찬가지로 [행렬을 인자로 사용하는 인터페이스][sm_wls]와 [formula를 사용하는 인터페이스][smf_wls]가 있다. "OLS"를 "WLS"로 바꾸고 "ols"를 "wls"로 바꾸면 된다. `weights`를 주는 것이 필수적이다. 예를 들어 `df`에 `y`, `x1`, `x2`, `wgt` 변수가 있다면 다음과 같이 할 수 있다(이분산이 있는 경우 오차분산이 $var(u_i)=\sigma^2 h_i$라면 효율적 추정을 위한 가중치는 $1/h$이다).

```python
import statsmodels.formula.api as smf
smf.wls("y ~ x1 + x2", data = df, weights = df["wgt"]).fit(cov_type = "HC1")
```

WLS에서 견고한 추론 문제는 [OLS의 경우][03]와 똑같이 적용된다.

[01]: 01%20Introduction%20to%20OLS.ipynb
[03]: 03%Robust%20Inferences%20with%20OLS.ipynb
[sm]: https://www.statsmodels.org/
[sm_wls]: https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.WLS.html
[smf_wls]: https://www.statsmodels.org/dev/generated/statsmodels.formula.api.wls.html
[het]: https://en.wikipedia.org/wiki/Homoscedasticity_and_heteroscedasticity

### 예제
지역별 집계(평균) 자료에서 오차항의 분산은 지역 인구에 반비례하는 것으로 설정할 수 있다. 즉, $\mathrm{var}(u_i) = \sigma^2 / pop_i$이다. 이 경우 $pop_i$를 가중치로 하는 WLS를 수행할 수 있다. 먼저 데이터를 읽어들이자.

In [7]:
import pandas as pd
import statsmodels.formula.api as smf

df = pd.read_csv('data/Death.csv')

OLS는 다음과 같이 하였다.

In [8]:
fm = 'deathrate ~ drink + smoke + aged + vehipc + C(year)'
ols = smf.ols(fm, data = df).fit(cov_type = 'cluster', cov_kwds={'groups':df["region"]})
print(ols.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:              deathrate   R-squared:                      0.9209
Model:                            OLS   Adj. R-squared:                 0.9190
No. Observations:                 258   F-statistic:                     493.2
Covariance Type:              cluster   Prob (F-statistic):             0.0000
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         -0.22413    1.01916      -0.22      0.826    -2.22165     1.77340
C(year)[T.2009]   -0.37876    0.07655      -4.95      0.000    -0.52879    -0.22873
C(year)[T.2010]   -0.35100    0.09895      -3.55      0.000    -0.54493    -0.15706
drink              0.00639    0.01410       0.45      0.650    -0.02125     0.03404
smoke              0.03328    0.01939       1.72      0.086    -0.00473     0.07128
aged             

WLS는 다음과 같이 하는데, 표준오차를 어떻게 구할지 생각해 보아야 한다. 먼저 $\mathrm{var}(u_i) = \sigma^2 / pop_i$ 설정이 맞다는 가정하에 통상적인 표준오차를 사용할 수 있다(이 통상적인 표준오차는 분산에 관한 가정이 맞다면 타당하고 분산에 관한 가정이 맞지 않다면 타당하지 않다). 이 경우라면 다음과 같이 한다.

In [9]:
wls = smf.wls(fm, data=df, weights=df.regpop)
wls_o = wls.fit()  # ordinary standard errors
print(wls_o.summary(slim=True))

                            WLS Regression Results                            
Dep. Variable:              deathrate   R-squared:                      0.9407
Model:                            WLS   Adj. R-squared:                 0.9393
No. Observations:                 258   F-statistic:                     663.9
Covariance Type:            nonrobust   Prob (F-statistic):             0.0000
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         -0.58150    0.76443      -0.76      0.448    -2.08700     0.92401
C(year)[T.2009]   -0.29631    0.09581      -3.09      0.002    -0.48501    -0.10761
C(year)[T.2010]   -0.29773    0.09897      -3.01      0.003    -0.49265    -0.10281
drink              0.01656    0.01052       1.57      0.117    -0.00415     0.03727
smoke              0.03267    0.01786       1.83      0.068    -0.00249     0.06784
aged             

오차 분산 구조가 설정한 바와 정확히 일치하고 오차 간 공분산이 0이라면 위의 통상적인 표준오차를 사용하는 것이 타당하나, 오차 분산 구조가 가정과 다르거나 오차 내 자기상관이 있다면 견고한 표준오차를 사용하여야 할 것이다. 위 자료는 패널 자료이므로 동일 지역 내에 자기상관이 있을 것으로 예상되고, 이를 고려하여 클러스터 표준오차를 사용하면 결과는 다음과 같다.

In [10]:
wls_c = wls.fit(cov_type="cluster", cov_kwds={'groups':df["region"]})
print(wls_c.summary(slim=True))

                            WLS Regression Results                            
Dep. Variable:              deathrate   R-squared:                      0.9407
Model:                            WLS   Adj. R-squared:                 0.9393
No. Observations:                 258   F-statistic:                     656.0
Covariance Type:              cluster   Prob (F-statistic):             0.0000
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         -0.58150    1.36200      -0.43      0.669    -3.25097     2.08798
C(year)[T.2009]   -0.29631    0.07409      -4.00      0.000    -0.44153    -0.15109
C(year)[T.2010]   -0.29773    0.09768      -3.05      0.002    -0.48918    -0.10627
drink              0.01656    0.02207       0.75      0.453    -0.02669     0.05981
smoke              0.03267    0.02195       1.49      0.137    -0.01034     0.07568
aged             

## 가중치를 추정하는 WLS

이분산 존재 시 FGLS를 위해 가중치를 추정하는 것을 고려하자. Wooldridge 교과서 *Introductory Econometrics*의 8-4b절 방법을 구현해 보자. 계량경제 모형이 주어질 때 분산 함수를 다음과 같이 추정한다.
1. 종속변수를 독립변수들에 대하여 OLS 회귀하여 잔차 $\hat{u}$을 구한다.
2. OLS 잔차들을 제곱한 후 로그를 취하여 $\log(\hat{u}^2)$을 만든다.
3. $\log(\hat{u}^2)$을 1번 모형의 독립변수들에 대하여 회귀하여 예측값 $\hat{g}$를 구한다.
4. $\hat{g}$를 지수함수 변환하여 $\hat{h} = \exp(\hat{g})$를 만든다.
5. $1/\hat{h}$를 가중치로 사용하여 원래 회귀식을 WLS 추정한다.

### 예제

이하에서 Wooldridge 교과서 예제 8.7을 복제해 보자.

In [11]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

df = pd.read_csv('data/smoke.csv')

fm = 'cigs ~ lincome + lcigpric + educ + age + agesq + restaurn'
ols = smf.ols(fm, data=df).fit()
df['uhat'] = ols.resid
fm_aux = 'np.log(uhat**2) ~ lincome + lcigpric + educ + age + agesq + restaurn'
aux = smf.ols(fm_aux, data=df).fit()
df['h'] = np.exp(aux.fittedvalues)
wls = smf.wls(fm, data=df, weights = 1/df['h'])
print(wls.fit(cov_type = "HC1").summary(slim=True))

                            WLS Regression Results                            
Dep. Variable:                   cigs   R-squared:                      0.1134
Model:                            WLS   Adj. R-squared:                 0.1068
No. Observations:                 807   F-statistic:                     23.56
Covariance Type:                  HC1   Prob (F-statistic):             0.0000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     5.63546   37.32339       0.15      0.880   -67.51704    78.78796
lincome       1.29524    0.53509       2.42      0.015     0.24647     2.34401
lcigpric     -2.94031    8.97045      -0.33      0.743   -20.52207    14.64144
educ         -0.46345    0.14906      -3.11      0.002    -0.75560    -0.17129
age           0.48195    0.11499       4.19      0.000     0.25657     0.70733
agesq        -0.00563    0.00118      -4.78      0.0

이 결과는 Wooldridge 교과서 예제 8.7 결과와 동일하다.

주석 \[2\]에서 condition number에 대하여 이야기하고 있으나 이것은 설명변수들 간에 상관이 높을 수 있음을 의미하는 것일 뿐이며, 공선성을 체크하면 문제 없는 것으로 나온다.

In [12]:
wls = smf.wls(fm, data=df, weights='_auto')
print(wls.fit(cov_type = "HC1").summary(slim=True))

                            WLS Regression Results                            
Dep. Variable:                   cigs   R-squared:                      0.1134
Model:                            WLS   Adj. R-squared:                 0.1068
No. Observations:                 807   F-statistic:                     23.56
Covariance Type:                  HC1   Prob (F-statistic):             0.0000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     5.63546   37.32339       0.15      0.880   -67.51704    78.78796
lincome       1.29524    0.53509       2.42      0.015     0.24647     2.34401
lcigpric     -2.94031    8.97045      -0.33      0.743   -20.52207    14.64144
educ         -0.46345    0.14906      -3.11      0.002    -0.75560    -0.17129
age           0.48195    0.11499       4.19      0.000     0.25657     0.70733
agesq        -0.00563    0.00118      -4.78      0.0